In [39]:
import pandas as pd

In [40]:
df=pd.read_csv("spotify_millsongdata.csv")

In [41]:
df.head(2)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."


In [42]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [43]:
df.duplicated().sum()

0

In [44]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [45]:
df.head()

,artist,song,text
0,Toto,Rockmaker,"Don't know quite what to say, haven't seen you..."
1,Dave Matthews Band,Shake Me Like A Monkey,The thing I like about you \r\nThe way you do...
2,Britney Spears,Prince Charming,Verse 1: \r\nHey Prince Charming \r\nYou can...
3,Reo Speedwagon,In Your Letter,In your letter ooh in your letter \r\nIn your...
4,Cher,Don't Come Cryin' To Me,Throw out our love \r\nLike it was old news ...


In [46]:
df.shape

(5000, 3)

In [47]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [48]:
def token(txt):
    token= nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [49]:
df['text'].apply(lambda x:token(x))

0       do n't know quit what to say , have n't seen y...
1       the thing i like about you the way you do what...
2       vers 1 : hey princ charm you can kiss my ass i...
3       in your letter ooh in your letter in your lett...
4       throw out our love like it wa old news be sure...
                              ...                        
4995    the kid will make it happen we 're start a new...
4996    who wa it organ the right to follow my leader ...
4997    here come mr. ' i ai n't like that ' , mr. 'yo...
4998    could i wake up next to you when it 's hit dou...
4999    oh babi babi , i stay in love with you die ins...
Name: text, Length: 5000, dtype: object

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [51]:
tfidf=TfidfVectorizer(analyzer='word',stop_words='english')

In [52]:
matrics=tfidf.fit_transform(df['text'])

In [53]:
similar=cosine_similarity(matrics)

In [64]:
def recommender(songs_name):
    idx=df[df['song']==songs_name].index[0]
    distance=sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [65]:
recommender("In Your Eyes")

['The One', 'Dreidel', 'Coming Down Again', 'What Happened Here']

In [66]:
import pickle

In [67]:
pickle.dump(similar,open("similarity","wb"))

In [68]:
pickle.dump(df,open("df","wb"))